IMPORTAMOS LIBRERÍAS

In [2]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

IMPORTAMOS EL CSV DE UBEREATS QUE LIMPIAREMOS

In [31]:
ue = pd.read_csv("../Data/UberEats.csv", index_col = 0)
ue.head(3)

,Restaurante,Valoración,Nota,Delivery_Fee,ETA,Tag,Precio,Dirección,Barrio
0,Badsha India Restaurant,200,4.4,1.00,15–35,India,€,"Calle De Embajadores, 28045",Centro
1,TUTTO PASTA!,26,2.8,2.50,40–60,Family Meals,€,"Calle De Núñez De Arce, 11, 28012",Centro
2,Pizza Central,31,3.8,2.99,35–55,Pizza,€,"Plaza Del Emperador Carlos V, 10, 28012",Centro


In [32]:
for col in ue:
    print(col)
    print(" ")
    print(ue[col].value_counts())
    print("-" * 30)

Restaurante
 
Empanadas BAIRES    13
Papa John's         13
McDonald's          12
Popeyes             11
Arepish             11
                    ..
Popizza              1
La Manousheria       1
Burger Las Vegas     1
Crepes & Waffles     1
Filippo Pizza        1
Name: Restaurante, Length: 251, dtype: int64
------------------------------
Valoración
 
200    84
23     48
30     38
15     37
40     37
       ..
194     1
78      1
145     1
63      1
80      1
Name: Valoración, Length: 82, dtype: int64
------------------------------
Nota
 
4.3    60
4.2    60
4.6    56
3.5    53
4.4    51
3.1    48
4.5    44
2.9    43
4.1    42
3.6    42
3.8    36
3.0    22
4.8    21
2.7    21
4.0    17
3.9    16
3.7    13
4.9    12
2.8    11
4.7    11
5.0     4
3.2     4
3.4     2
3.3     1
Name: Nota, dtype: int64
------------------------------
Delivery_Fee
 
2.40    165
1.90    131
1.40    122
0.90     85
2.90     67
2.50     27
0.40     19
1.00     12
3.00     12
4.25     11
3.40     11
2.99      